In [1]:
import os
import pandas as pd
import polars as pl
pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

%load_ext autoreload
%autoreload 2

In [19]:
import os
from pathlib import Path

import polars as pl

from EventStream.data.dataset_polars import Dataset

DATA_DIR = Path("../MIMIC_ESD_new_schema_08-31-23-1")
ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.filter(~pl.all_horizontal(pl.all().is_null()))
dynamic_measurements_df = ESD.dynamic_measurements_df.filter(~pl.all_horizontal(pl.all().is_null()))

df_data = (events_df
            .join(dynamic_measurements_df, on="event_id", how="left")
            .drop(['event_id'])
            .sort(by=['subject_id', 'timestamp', 'event_type'])
            )


Updating config.save_dir from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_new_schema_08-31-23-1 to ../MIMIC_ESD_new_schema_08-31-23-1
Loading events from ../MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet...
Loading dynamic_measurements from ../MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet...


### End-to-End

In [31]:
from task_querying_v2 import *

name = 'test_07'

config_path = f"test_configs/{name}.yaml"

res = query_task(config_path, df_data)
res

Loading config...

Generating predicate columns...

Added predicate column is_admission.
Added predicate column is_discharge.
Added predicate column is_death.
Added predicate column is_covid.
Added predicate column is_lab.
Added predicate column is_discharge_or_death.

Building tree...
trigger
┣━━ gap
┃   ┗━━ target
┗━━ input


Querying...
Querying subtree rooted at gap...
Querying subtree rooted at target...
Querying subtree rooted at input...
Done.



subject_id,trigger/timestamp,gap/timestamp,target/timestamp,input/timestamp,gap/window_summary,target/window_summary,input/window_summary,label
u16,datetime[μs],datetime[μs],datetime[μs],datetime[μs],struct[7],struct[7],struct[7],i32
0,2130-10-08 19:03:00,2130-10-10 19:03:00,2130-10-12 19:05:00,2130-11-08 19:03:00,"{0,0,0,0,0,0,0}","{1,1,0,0,1,0,2}","{2,3,0,0,3,0,5}",0
0,2130-12-27 21:23:00,2130-12-29 21:23:00,2130-12-30 15:33:00,2131-01-27 21:23:00,"{0,0,0,0,0,0,0}","{1,1,0,0,1,0,2}","{1,2,1,0,2,11917,1017}",0
0,2131-01-07 20:39:00,2131-01-09 20:39:00,2131-01-20 05:15:00,2131-02-07 20:39:00,"{0,0,0,0,0,0,0}","{1,1,1,0,1,11917,1010}","{0,1,1,0,1,11917,1015}",1
1,2129-08-04 12:44:00,2129-08-06 12:44:00,2129-08-18 16:53:00,2129-09-04 12:44:00,"{0,0,0,0,0,1493,124}","{1,1,0,0,1,2960,243}","{0,1,0,0,1,3578,285}",0
1,2130-09-23 21:59:00,2130-09-25 21:59:00,2130-09-29 18:55:00,2130-10-24 21:59:00,"{0,0,0,0,0,1336,113}","{1,1,0,0,1,1179,102}","{0,1,0,0,1,1822,148}",0
2,2156-04-12 14:16:00,2156-04-14 14:16:00,2156-04-29 16:26:00,2156-05-13 14:16:00,"{0,0,0,0,0,1923,222}","{1,1,0,0,1,10465,918}","{2,2,0,0,2,15631,1435}",0
2,2156-04-30 20:35:00,2156-05-02 20:35:00,2156-05-03 16:36:00,2156-05-31 20:35:00,"{0,0,0,0,0,1607,159}","{1,1,0,0,1,9,7}","{1,2,0,0,2,12010,989}",0
2,2156-05-11 14:49:00,2156-05-13 14:49:00,2156-05-22 14:16:00,2156-06-11 14:49:00,"{0,0,0,0,0,2650,254}","{1,1,0,0,1,8609,659}","{0,1,0,0,1,9929,776}",0
3,2137-02-07 19:42:00,2137-02-09 19:42:00,2137-02-18 18:30:00,2137-03-10 19:42:00,"{0,0,0,0,0,0,0}","{1,1,0,0,1,0,2}","{1,1,0,0,1,14610,1399}",0


In [10]:
res.select(['target/window_summary']).unnest('target/window_summary')

# Unit Testing

In [40]:
from simple_test_runner import *

In [41]:
df_test_1_input = pl.DataFrame(
    {
        'subject_id': [1, 1, 1],
        'timestamp': ['12/1/1900 12:00', '12/1/1900 13:00', '12/1/1900 14:00'],
        'event_type': ['ADMISSION', 'LAB', 'DISCHARGE'],
        'dx': ['', '', ''],
        'lab_test': ['', 'SpO2', ''],
        'lab_value': ['', '99', ''],
        'is_admission': [1, 0, 0],
        'is_lab': [0, 1, 0],
        'is_discharge': [0, 0, 1],
        'is_any': [1, 1, 1],
    }
)

df_test_1_predicate_cols = [col for col in df_test_1_input.columns if col.startswith('is_')]

df_test_1_endpoint_expr = (False, "is_discharge", True, None) 

test_1_anchor_to_subtree_root_by_subtree_anchor = pl.DataFrame(
    {
        'subject_id': [1],
        'timestamp': ['12/1/1900 12:00'],
        'is_admission': [0],
        'is_lab': [0],
        'is_discharge': [0],
        'is_any': [0],
    }
)

df_test_1_result = pl.DataFrame(
    {
        "subject_id": [1],
        "timestamp": ["12/1/1900 14:00"],
        "timestamp_at_anchor": ["12/1/1900 12:00"],
        "is_admission": [0],
        "is_lab": [1],
        "is_discharge": [1],
        "is_any": [2],
        "is_admission_summary": [0],
        "is_lab_summary": [0],
        "is_discharge_summary": [0],
        "is_any_summary": [0],
    }
)

In [43]:
test_1 = [
    {
        "message": "Testing summarize_event_bound_window, should be equal...",
        "args": (
            df_test_1_input, 
            df_test_1_predicate_cols, 
            df_test_1_endpoint_expr,
            test_1_anchor_to_subtree_root_by_subtree_anchor),
        "want": df_test_1_result,
    }
]

simple_test_runner(test_1, summarize_event_bound_window)

Testing summarize_event_bound_window, should be equal... Passed!


# Integration Testing

### Config Loading

In [12]:
from task_querying_v2 import *

In [13]:
data = pl.read_csv('sample_data.csv')
data = data.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, format='%m/%d/%Y %H:%M').cast(pl.Datetime))

In [14]:
cfg = load_config('sample_config.yaml')
cfg

{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Generate predicate columns

In [15]:
predicates_df = generate_predicate_columns(cfg, data)
predicates_df

Added predicate column is_admission.
Added predicate column is_discharge.
Added predicate column is_death.
Added predicate column is_covid.
Added predicate column is_discharge_or_death.


subject_id,timestamp,event_type,dx,lab_test,lab_value,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any
i64,datetime[μs],str,str,str,i64,i32,i32,i32,i32,i32,i32
1,1989-12-01 12:03:00,"""ADMISSION""",null,null,null,1,0,0,0,0,1
1,1989-12-01 13:14:00,"""LAB""",null,"""SpO2""",99,0,0,0,0,0,1
1,1989-12-01 15:17:00,"""LAB""",null,"""SpO2""",98,0,0,0,0,0,1
1,1989-12-01 16:17:00,"""LAB""",null,"""SpO2""",99,0,0,0,0,0,1
1,1989-12-01 20:17:00,"""LAB""",null,"""SpO2""",98,0,0,0,0,0,1
1,1989-12-02 03:00:00,"""LAB""",null,"""SpO2""",99,0,0,0,0,0,1
1,1989-12-02 09:00:00,"""DIAGNOSIS""","""FLU""",null,null,0,0,0,0,0,1
1,1989-12-02 15:00:00,"""DISCHARGE""",null,null,null,0,1,0,0,1,1
1,1991-01-27 23:32:00,"""ADMISSION""",null,null,null,1,0,0,0,0,1


### Build tree

In [16]:
tree = build_tree_from_config(cfg)
print_tree(tree, style="const_bold")
for each_node in preorder_iter(tree):
    print(each_node)
cfg

trigger
┣━━ gap
┃   ┗━━ target
┗━━ input
Node(/trigger, constraints={'is_admission': (1, 1)}, endpoint_expr=(False, 'is_admission', True, datetime.timedelta(0)))
Node(/trigger/gap, constraints={'is_admission': (None, 0), 'is_discharge': (None, 0), 'is_death': (None, 0)}, endpoint_expr=(False, datetime.timedelta(days=2), True, datetime.timedelta(0)))
Node(/trigger/gap/target, constraints={'is_discharge_or_death': (1, 1)}, endpoint_expr=(True, 'is_discharge_or_death', True, datetime.timedelta(0)))
Node(/trigger/input, constraints={'is_covid': (None, 0), 'is_any': (5, None)}, endpoint_expr=(False, datetime.timedelta(days=30), True, datetime.timedelta(days=1)))


{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Summarize window

In [17]:
import datetime

predicate_cols = [col for col in predicates_df.columns if col.startswith("is_")]
print(predicate_cols)

anchor_to_subtree_root_by_subtree_anchor = (
        predicates_df.filter(predicates_df['is_admission'] == 1)
            .select('subject_id', 'timestamp', *[pl.col(c) for c in predicate_cols])
            .with_columns('subject_id', 'timestamp', *[pl.lit(0).alias(c) for c in predicate_cols])
    )
print(anchor_to_subtree_root_by_subtree_anchor)

['is_admission', 'is_discharge', 'is_death', 'is_covid', 'is_discharge_or_death', 'is_any']
shape: (6, 8)
┌────────────┬─────────────┬─────────────┬─────────────┬──────────┬──────────┬────────────┬────────┐
│ subject_id ┆ timestamp   ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_covid ┆ is_dischar ┆ is_any │
│ ---        ┆ ---         ┆ n           ┆ e           ┆ ---      ┆ ---      ┆ ge_or_deat ┆ ---    │
│ i64        ┆ datetime[μs ┆ ---         ┆ ---         ┆ i32      ┆ i32      ┆ h          ┆ i32    │
│            ┆ ]           ┆ i32         ┆ i32         ┆          ┆          ┆ ---        ┆        │
│            ┆             ┆             ┆             ┆          ┆          ┆ i32        ┆        │
╞════════════╪═════════════╪═════════════╪═════════════╪══════════╪══════════╪════════════╪════════╡
│ 1          ┆ 1989-12-01  ┆ 0           ┆ 0           ┆ 0        ┆ 0        ┆ 0          ┆ 0      │
│            ┆ 12:03:00    ┆             ┆             ┆          ┆          ┆        

Temporally-bound

In [18]:
summarize_temporal_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, timedelta(days=2), True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-01 12:03:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-27 23:32:00,1991-01-27 23:32:00,0,0,0,0,0,3,0,0,0,0,0,0
1,1991-03-03 19:33:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 02:24:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-05 00:32:00,1996-06-05 00:32:00,0,0,0,0,0,0,0,0,0,0,0,0
3,1996-03-08 02:24:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0


Event-bound

In [19]:
summarize_event_bound_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, 'is_discharge_or_death', True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-02 15:00:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-31 02:15:00,1991-01-27 23:32:00,0,1,0,0,1,9,0,0,0,0,0,0
1,1991-03-03 21:38:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 16:00:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-08 03:00:00,1996-06-05 00:32:00,0,0,1,0,1,5,0,0,0,0,0,0
3,1996-03-10 00:00:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0
